### Import Libraries

In [ ]:
# Libraries
import pandas as pd
import numpy as np
from datetime import date
import warnings
warnings.filterwarnings("ignore")
import mysql.connector
from sqlalchemy import create_engine
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import matplotlib.pyplot as plt

In [ ]:
# Set Up Selenium
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

options = Options()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('start-maximized')
options.add_argument('disable-infobars')
options.add_argument("--disable-extensions")

prefs = {"profile.managed_default_content_settings.images": 2}
options.add_experimental_option("prefs", prefs)

driver = webdriver.Chrome('/usr/local/bin/chromedriver', options=options)

timeout = 3

### Get Data

In [ ]:
# Database connection
cnx = mysql.connector.connect(user='admin',  password='password',
                              host='db-instance.rds.amazonaws.com',
                              port= 3306,
                              database='My_DB',
                              use_unicode=True)
                              
cursor=cnx.cursor(buffered=True)
engine = create_engine('mysql+mysqlconnector://{0}:{1}@{2}/{3}'.
                                               format('admin', 'password', 
                                                      'db-instance.rds.amazonaws.com:3306', 'My_DB'))
# Get Booked Hotel Data
sql = "SELECT * FROM My_DB.Hotel_Booked_DB;"
HotelBookedDF = pd.read_sql(sql, con=cnx)
HotelBookedDF = HotelBookedDF.sort_values('Current',ascending=True)

# Get Hotel Data
sql = "SELECT * FROM My_DB.Hotel_DB;"
HotelDF = pd.read_sql(sql, con=cnx)
HotelDF.drop('ID', axis=1, inplace=True)
HotelDF.head(2)

In [ ]:
# Download Hotel Prices
HotelDF['Price'] = 0
for index in range(0,len(HotelDF)):
    try:
        URL = "https://www.booking.com/hotel/us/" + HotelDF['Hotel_ID'][index] + ".en-gb.html?aid=304142;checkin=2020-12-25;checkout=2020-12-30;dest_id=20088325;dest_type=city;dist=0;group_adults=2;group_children=0; no_rooms=1;room1=A%2CA;sb_price_type=total;type=total;ucfs=1&"

        driver.get(URL)
        Prices = driver.find_elements_by_class_name('hprt-table')[0].find_elements_by_class_name('per-night-tt-total-price')
        Conditions = driver.find_elements_by_class_name('hprt-table')[0].find_elements_by_class_name('hprt-conditions')

        for index2 in range(0,len(Prices)):
            if 'Non-refundable' in Conditions[index2].get_attribute('innerHTML'):
                HotelDF['Price'].loc[index] = None
            else:
                HotelDF['Price'].loc[index] = Prices[index2].get_attribute('innerHTML').replace('£', '').replace(',', '')
                break
    except:
        print("Full: " + HotelDF['Hotel_Name'][index])
    print(str(index) + "/" + str(len(HotelDF)) + "                      ", end='\r')

# View Results
HotelDF['Price'] = pd.to_numeric(HotelDF['Price']) 
HotelDF['Price'].replace(0, np.nan, inplace=True)
HotelDF = HotelDF.sort_values('Price',ascending=True)
HotelDF.reset_index(drop=True, inplace=True)       
HotelDF.head()    

### Upload Results

In [ ]:
# Save Results to DB
resultsDF = HotelDF[['Hotel_ID', 'Price']]
resultsDF['Date'] = str(date.today())
resultsDF.to_sql(con=engine, name='Hotel_Prices_DB', if_exists='append',index=False)

### Send Notifications

In [ ]:
# Set up email config
import smtplib

s = smtplib.SMTP('smtp.email.com', 587) # Define Server
s.starttls() # Start TLS
s.login("email", "password") # Log in to server

# Send Email IF cheap price
if HotelDF['Price'][0] < HotelBookedDF['Price'][0]:
    # found a cheaper hotel
    message = "Cheaper Hotel found: \n Current hotel price: £" + str(HotelBookedDF['Price'][0]) +" \n " +\
              "New hotel price: £" + str(HotelDF['Price'][0]) + "\n" +\
              "Hotel: " + HotelDF['Hotel_Name'][0] + "\n" +\
              "URL: " + HotelDF['Hotel_URL'][0]
    
    parts = ("From: " + 'email',
         "To: " + 'email',
         "Subject: " + "Hotel Prices API",
         "",
         message)    
    msg = '\r\n'.join(parts).encode('ascii', 'ignore')
    
    # Send email
    s.sendmail("email", "email", msg)
    

## Visualisations

In [ ]:
# Get Hotel Data
sql = "SELECT * FROM My_DB.Hotel_Prices_DB;"
HotelPriceDF = pd.read_sql(sql, con=cnx)

# Convert date type
HotelPriceDF['Date'] =  pd.to_datetime(HotelPriceDF['Date'], format="%Y/%m/%d")

# Replace NaN values
HotelPriceDF = HotelPriceDF.fillna(0)

# Set Date as Index
HotelPriceDF = HotelPriceDF.set_index('Date')

# Visualise
plt.figure(figsize=(20,15))
HotelPriceDF.groupby('Hotel_ID')['Price'].plot()
plt.title("Hotel Prices", fontdict={'fontsize': 19, 'fontweight':0.5 })
plt.xlabel("Date", fontdict={'fontsize': 15})
plt.ylabel("Price (£)", fontdict={'fontsize': 15})
plt.gca().legend((HotelPriceDF['Hotel_ID']), bbox_to_anchor=(1.0, 1.0))
plt.show()